## <center>**Sincronização de Banco de Dados** - Grupo 2<center>


Imports

In [16]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

True

Conexão com o Banco Origem 

In [17]:
conn_source = psycopg2.connect(
    host=os.getenv("SOURCE_HOST"),
    database="db Copa ",
    user=os.getenv("SOURCE_USER"),
    password=os.getenv("SOURCE_PASSWORD"),
    port=os.getenv("SOURCE_PORT")
)

try:
    cur_source = conn_source.cursor()
    print("Conexão estabelecida com sucesso!")
except Exception as e:
    print("Erro ao conectar ao banco de dados:\n", e)

Conexão estabelecida com sucesso!


Conexão com o Banco Target

In [18]:
conn_target = psycopg2.connect(
    host=os.getenv("TARGET_HOST"),
    database=os.getenv("TARGET_DATABASE"),
    user=os.getenv("TARGET_USER"),
    password=os.getenv("TARGET_PASSWORD"),
    port=os.getenv("TARGET_PORT")
)

try:
    cur_target = conn_target.cursor()
    print("Conexão estabelecida com sucesso!")
except Exception as e:
    print("Erro ao conectar ao banco de dados:\n", e)

Conexão estabelecida com sucesso!


Funções de INSERT

In [19]:
def insert_paises(df, cur_target, conn_target):
    query = """
        INSERT INTO paises (
            id, nome, continente, qnt_titulos,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
    """
    data = [
        (
            int(ii["id"]),
            ii["nome"],
            ii["continente"],
            int(ii["qnt_titulos"]),
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

In [20]:
def insert_jogadores(df, cur_target, conn_target):
    query = """
        INSERT INTO jogadores (
            id, nome, data_nascimento, posicao, altura, peso, pais_id,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
    """
    data = [
        (
            int(ii["id"]),
            ii["nome"],
            ii["data_nascimento"],
            ii["posicao"],
            float(ii["altura"]),
            float(ii["peso"]),
            int(ii["pais_id"]),
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()


In [21]:
def insert_partidas(df, cur_target, conn_target):
    query = """
        INSERT INTO partidas (
            id, data_partida, estadio, time_casa_id, time_visitante_id,
            placar_casa, placar_visitante, qnt_faltas_casa, qnt_faltas_visitante,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
    """
    data = [
        (
            int(ii["id"]),
            ii["data_partida"],
            ii["estadio"],
            int(ii["time_casa_id"]),
            int(ii["time_visitante_id"]),
            int(ii["placar_casa"]),
            int(ii["placar_visitante"]),
            int(ii["qnt_faltas_casa"]),
            int(ii["qnt_faltas_visitante"]),
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

In [22]:
def insert_campeonatos(df, cur_target, conn_target):
    query = """
        INSERT INTO campeonatos (
            id, ano, sede, campeao_id, vice_id,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
    """
    data = [
        (
            int(ii["id"]),
            int(ii["ano"]),
            ii["sede"],
            ii["campeao_id"],
            ii["vice_id"],
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()


In [23]:
def insert_fases(df, cur_target, conn_target):
    query = """
        INSERT INTO fases (
            id, nome,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
    """
    data = [
        (
            int(ii["id"]),
            ii["nome"],
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

In [24]:
def insert_chaveamento(df, cur_target, conn_target):
    query = """
        INSERT INTO chaveamento (
            id, campeonato_id, fase_id, partida_id, vencedor_id,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
    """
    data = [
        (
            int(ii["id"]),
            int(ii["campeonato_id"]),
            int(ii["fase_id"]),
            int(ii["partida_id"]),
            ii["vencedor_id"],
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

Funções de UPDATE

In [25]:
def update_paises(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE paises
            SET nome = %s,
                continente = %s,
                qnt_titulos = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["nome"],
            ii["continente"],
            ii["qnt_titulos"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [26]:
def update_jogadores(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE jogadores
            SET nome = %s,
                data_nascimento = %s,
                posicao = %s,
                altura = %s,
                peso = %s,
                pais_id = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["nome"],
            ii["data_nascimento"],
            ii["posicao"],
            ii["altura"],
            ii["peso"],
            ii["pais_id"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [27]:
def update_partidas(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE partidas
            SET data_partida = %s,
                estadio = %s,
                time_casa_id = %s,
                time_visitante_id = %s,
                placar_casa = %s,
                placar_visitante = %s,
                qnt_faltas_casa = %s,
                qnt_faltas_visitante = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["data_partida"],
            ii["estadio"],
            ii["time_casa_id"],
            ii["time_visitante_id"],
            ii["placar_casa"],
            ii["placar_visitante"],
            ii["qnt_faltas_casa"],
            ii["qnt_faltas_visitante"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [28]:
def update_campeonatos(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE campeonatos
            SET ano = %s,
                sede = %s,
                campeao_id = %s,
                vice_id = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["ano"],
            ii["sede"],
            ii["campeao_id"],
            ii["vice_id"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [29]:
def update_fases(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE fases
            SET nome = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["nome"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [30]:
def update_chaveamento(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE chaveamento
            SET campeonato_id = %s,
                fase_id = %s,
                partida_id = %s,
                vencedor_id = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["campeonato_id"],
            ii["fase_id"],
            ii["partida_id"],
            ii["vencedor_id"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()